# GP2GP Failure Rate - Oct 2020

Context:
- PCSE wants the rate of paper transfers being triggered in particular, over the last three months.
- Finding the totals and rates of failures involved calculating, for each month: 
    - the total number of transfers
    - the total number of successful transfers completed within 8 days SLA 
    - the total number of successful transfers completed beyond 8 days SLA
    - the total remaining number of transfers assumed unsuccessful

Assumptions:
- We define the failure rate as a transfer without a successful final ack, an incorrect message sequence or not within the 8 day SLA.
- We are aware that there are transfers where a message could contain a placeholder (therefore triggering the paper process), however since this metadata is not provided in the Spine logs, we can not identify these failures and they will be classified as complete.

Requirements:
- This notebook uses the following Splunk query:
```
index="spine2vfmmonitor" service="gp2gp"
| search interactionID="urn:nhs:names:services:gp2gp/*"
| rex field=fromPartyID "(?<fromNACS>.+)(-\d*)"
| rex field=toPartyID "(?<toNACS>.+)(-\d*)"
| fields _time, conversationID, GUID, interactionID, fromNACS, toNACS, messageRef, jdiEvent
```

In [2]:
from gp2gp.spine.models import COMMON_POINT_TO_POINT, EHR_REQUEST_COMPLETED
from sys import argv
from gp2gp.io.csv import read_gzip_csv_files
from gp2gp.spine.sources import construct_messages_from_splunk_items
from gp2gp.spine.transformers import (
    group_into_conversations,
    parse_conversation,
    filter_conversations_by_request_started_time,
    ConversationMissingStart,
)
from gp2gp.service.transformers import (
    derive_transfers,
    filter_for_successful_transfers,
    calculate_sla_by_practice,
)
from gp2gp.service.models import (
    SlaBand,
)
from gp2gp.pipeline.dashboard.core import (
    _parse_conversations
)
from gp2gp.date.range import DateTimeRange
from datetime import datetime
from dateutil.tz import UTC
from dateutil.relativedelta import relativedelta

In [3]:
def calculate_date_range(year, month):
    metric_month = datetime(year, month, 1, tzinfo=UTC)
    next_month = metric_month + relativedelta(months=1)
    time_range = DateTimeRange(metric_month, next_month)
    return time_range   

In [4]:
EIGHT_DAYS_IN_SECONDS = 691200

def assign_to_sla(sla_duration):
    sla_duration_in_seconds = sla_duration.total_seconds()
    if sla_duration_in_seconds <= EIGHT_DAYS_IN_SECONDS:
        return SlaBand.WITHIN_8_DAYS
    else:
        return SlaBand.BEYOND_8_DAYS


In [5]:
def calculate_sla(transfers):
    default_sla = {SlaBand.WITHIN_8_DAYS: 0, SlaBand.BEYOND_8_DAYS: 0}
    for transfer in transfers:
        sla_band = assign_to_sla_band(transfer.sla_duration)
        default_sla[sla_band] += 1
    return default_sla

In [ ]:
# Change below to point to location of zipped file
input_file_name = "../data/test_gp2gp_dec_2019.csv.gz"
csv_file_content = read_gzip_csv_files([input_file_name])
spine_messages = construct_messages_from_splunk_items(csv_file_content)

In [32]:
conversations = group_into_conversations(spine_messages)
parsed_conversations = _parse_conversations(conversations)
time_range = calculate_date_range(2019, 12)
conversations_started_in_range = filter_conversations_by_request_started_time(
    parsed_conversations, time_range
)
print(f"time_range: {time_range}")

In [33]:
transfers = derive_transfers(conversations_started_in_range)
alltransfers = list(transfers)
print(f"TOTAL TRANSFERS: {len(alltransfers)}")

completed_transfers = filter_for_successful_transfers(iter(alltransfers))
allcompleted = list(completed_transfers)
print(f"TOTAL COMPLETED TRANSFERS: {len(allcompleted)}")

slas = calculate_sla(allcompleted)
print(f"SLAS: {slas}")


Content extracted from gzip file
Messages constructed
Conversations constructed
Conversations parsed
time_range: <gp2gp.date.range.DateTimeRange object at 0x7f8b80cdbaf0>
---- Starting to transform data ----
TOTAL TRANSFERS: 1
TOTAL COMPLETED TRANSFERS: 0
SLAS: {<SlaBand.WITHIN_8_DAYS: 2>: 0, <SlaBand.BEYOND_8_DAYS: 3>: 0}
